# NY Food review project

This notebook contains testing and scratch work

### Imports

In [108]:
%load_ext autoreload
%autoreload 2

# Import ds libraries
import pandas as pd
import numpy as np
import re

# Import acquire functions
import nick_acquire as a
import nick_prepare as prep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
pd.set_option('display.max_columns', None)

### Data dictionary

|          feature          |                            description                           |
| ------------------------- | ---------------------------------------------------------------- |
| camis                     | Unique identifier for the restaurant                             |
| dba                       | Name of the business                                             |
| boro                      | Borough in which restaurant is located                           |
| building                  | Building number for restaurant                                   |
| street                    | Street name for establishment                                    |
| zipcode                   | Zip code for the establishment                                   |
| phone                     | Phone number for the establishment                               |
| inspection_date           | Date of the inspection of the restaurant                         |
| critical_flag             | Indicator of critical violation                                  |
| record_date               | The date when the extract was run to produce this data set       |
| latitude                  | Latitude                                                         |
| longitude                 | Longitude                                                        |
| community_board           | Local government body in the five boroughs of New York City      |
| council_district          | District of a New York City Council member                       |
| census_tract              | This is a geographic region  for the purpose of a census         |
| bin                       | This stands for Building Identification Number.                  |
| bbl                       | Borough, Block, and Lot. It's a unique real state id             |
| nta                       | Neighborhood Tabulation Area                                     |
| cuisine_description       | Describes type of cuisine at the restaurant                      |
| action                    | The actions that is associated with each restaurant inspection   |
| violation_code            | Violation code associated with establishment inspection          |
| violation_description     | Violation description associated with establishment inspection   |
| score                     | Total score for a particular inspection                          |
| grade                     | Grade associated with inspection                                 |
| grade_date                | Date when the current grade was issued                           |
| inspection_type           | Combination of the inspection program and the type of inspection |

This field represents the actions that is associated with each restaurant inspection. ; 

* Violations were cited in the following area(s). 
* No violations were recorded at the time of this inspection. 
* Establishment re-opened by DOHMH 
* Establishment re-closed by DOHMH 
* Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed. 
* "Missing" = not yet inspected;

In [4]:
ny = a.acquire_ny()
ny.head(3)

,camis,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,record_date,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,cuisine_description,action,violation_code,violation_description,score,grade,grade_date,inspection_type
0,50106756,UNGARO COAL FIRED PIZZA CAFE,Staten Island,1298,FOREST AVENUE,10302.0,6464690930,1900-01-01T00:00:00.000,Not Applicable,2023-10-26T06:00:14.000,40.626371,-74.133111,501.0,50.0,20100.0,5170408.0,5.003870e+09,SI07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50105716,STELLA'S,Brooklyn,559,5 AVENUE,11215.0,4155703174,1900-01-01T00:00:00.000,Not Applicable,2023-10-26T06:00:14.000,40.665416,-73.989417,307.0,39.0,14100.0,3337750.0,3.010480e+09,BK37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41168748,DUNKIN,Bronx,880,GARRISON AVENUE,10474.0,7188614171,2022-03-30T00:00:00.000,Not Critical,2023-10-26T06:00:11.000,40.816753,-73.892364,202.0,17.0,9300.0,2098685.0,2.027390e+09,BX27,Donuts,Violations were cited in the following area(s).,10J,Hand wash sign not posted,13.0,A,2022-03-30T00:00:00.000,Cycle Inspection / Initial Inspection


 ## Unique counts of columns within dataframe

In [4]:
ny.nunique()

camis                    28232
dba                      22114
boro                         6
building                  7479
street                    2403
zipcode                    226
phone                    25633
inspection_date           1678
critical_flag                3
record_date                  3
latitude                 23115
longitude                23115
community_board             69
council_district            51
census_tract              1183
bin                      20020
bbl                      19709
nta                        193
cuisine_description         89
action                       5
violation_code             143
violation_description      221
score                      130
grade                        6
grade_date                1455
inspection_type             31
dtype: int64

In [5]:
ny.camis.nunique()

28232

In [6]:
ny.dba.nunique()

22114

In [7]:
ny.isna().sum()

camis                         0
dba                         508
boro                          0
building                    351
street                        6
zipcode                    2680
phone                         7
inspection_date               0
critical_flag                 0
record_date                   0
latitude                    257
longitude                   257
community_board            3247
council_district           3251
census_tract               3251
bin                        4237
bbl                         573
nta                        3247
cuisine_description        2305
action                     2305
violation_code             3452
violation_description      3452
score                      9706
grade                    105753
grade_date               114506
inspection_type            2305
dtype: int64

In [8]:
ny_info = pd.DataFrame(ny.isna().sum())
ny_info['dtype'] = ny.dtypes
ny_info = ny_info.rename(columns={0:'nulls'})

In [9]:
ny_info.T

,camis,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,record_date,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,cuisine_description,action,violation_code,violation_description,score,grade,grade_date,inspection_type
nulls,0,508,0,351,6,2680,7,0,0,0,257,257,3247,3251,3251,4237,573,3247,2305,2305,3452,3452,9706,105753,114506,2305
dtype,int64,object,object,object,object,float64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,float64,object,object,object


In [10]:
len(ny)

207929

In [36]:
ny = a.acquire_ny()

### Drop useless columns

In [81]:
def remove_columns(ny , trash_columns = ['bin', 'bbl', 'nta', 'census_tract', 'council_district', 'community_board', 'grade_date']):
    ny = ny.drop(columns=trash_columns)
    return ny

In [37]:
trash_columns = ['bin', 'bbl', 'nta', 'census_tract', 'council_district', 'community_board', 'grade_date']
ny = ny.drop(columns=trash_columns)

### Clean phone numbers

In [6]:
def clean_phones(ny):    
    # Clean phone numbers by removing non-digit characters and dropping nulls
    ny.phone = ny.phone.str.replace(' ','')
    ny.phone = ny.phone.str.replace('_','')
    ny = ny[ny.phone.notna()]
    return ny

In [38]:
ny = clean_phones(ny)

### Clean zipcodes

In [7]:
def clean_zipcodes(ny):
    # Clean zipcodes by filling nulls with 0 and then converting to integers
    ny.zipcode = ny.zipcode.fillna(0)
    ny.zipcode = ny.zipcode.astype(int)
    ny = ny[ny.zipcode.notna()]  # Drop nulls
    return ny

In [39]:
ny = clean_zipcodes(ny)

### Clean streets

In [8]:
def clean_streets(ny):
    # Remove nulls from street
    ny = ny[ny.street.notna()]
    return ny

In [40]:
ny = clean_streets(ny)

### Clean scores

In [17]:
def clean_scores(data):
    ny = data.copy()
    ny = ny[ny.inspection_date != '1900-01-01T00:00:00.000']  # Remove all values with no inspections done
    
    # Create a new list of scores that replaces null scores for no violation for 0s
    new_scores = []  # Empty list
    for score,rep in zip(ny.score, ny.action.str.contains('No violation')):  # Loop through 2 iterable values
        if rep == True:  # If no violation, append score 0
            new_scores.append(0)
        else:  # Else keep score the same
            new_scores.append(score)
    ny.score = new_scores
    
    ny = ny[ny.score.notna()]
    return ny

In [41]:
ny = clean_scores(ny)

### Clean actions

In [56]:
def clean_actions(ny):
    # Remove nulls from action
    ny = ny[ny.action.notna()]
    # Rename actions to something more concise
    ny.action = np.where(ny.action == 'Violations were cited in the following area(s).', 'Violations cited', ny.action)
    ny.action = np.where(ny.action == 'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.', 'Closed', ny.action)
    ny.action = np.where(ny.action == 'Establishment re-opened by DOHMH.', 'Re-opened', ny.action)
    ny.action = np.where(ny.action == 'No violations were recorded at the time of this inspection.', 'No violations', ny.action)
    return ny

In [59]:
ny = clean_actions(ny)

### Clean grades

In [47]:
def clean_grades(data):
    ny = data.copy()  # Create copy of df
    # Create empty list to hold new values for restaurant
    new_grades = []
    # Use scores to determine grades
    for grade, score in zip(ny.grade, ny.score):
        if score <= 13:
            new_grades.append('A')
        elif score <= 27:
            new_grades.append('B')
        elif score > 27:
            new_grades.append('C')
    ny.grade = new_grades
    return ny

In [48]:
ny = clean_grades(ny)

### Clean violation code

In [73]:
def clean_violations(data):
    ny = data.copy()
    # Create empty lists
    new_codes = []
    new_description = []
    # Loop through actions and violations and if there is no violations in action, append no violations to code and description
    for action, code, description in zip(ny.action, ny.violation_code, ny.violation_description):
        if action == 'No violations':
            new_codes.append('No violation')
            new_description.append('No violation')
        else:
            new_codes.append(code)
            new_description.append(description)
            
    # Replace df values with new ones
    ny.violation_code = new_codes
    ny.violation_description = new_description

    return ny  # Return data

In [74]:
ny = clean_violations(ny)

### Everything else

In [77]:
ny = ny.dropna()

In [78]:
ny.isna().sum()

camis                    0
dba                      0
boro                     0
building                 0
street                   0
zipcode                  0
phone                    0
inspection_date          0
critical_flag            0
record_date              0
latitude                 0
longitude                0
cuisine_description      0
action                   0
violation_code           0
violation_description    0
score                    0
grade                    0
inspection_type          0
dtype: int64

In [79]:
len(ny)

198289

In [82]:
def clean_ny(ny):
    
    """This function just takes in all other cleaning functions for ny data and cleans each element of it"""
    
    ny = remove_columns(ny)  # Removes useless columns from ny health inspection data
    
    ny = clean_phones(ny)  # Clean phone numbers
    
    ny = clean_zipcodes(ny)  # Cleans zip codes
    
    ny = clean_streets(ny)  # Cleans streets
    
    ny = clean_scores(ny)  # Cleans scores
    
    ny = clean_actions(ny)  # Cleans actions
    
    ny = clean_grades(ny)  # Cleans grades
    
    ny = clean_violations(ny)  # Cleans violation codes and descriptions
    
    ny = ny.dropna()  # Drops all remaining null values
    
    return ny  # Return clean dataframe

In [83]:
ny = a.acquire_ny()
ny = clean_ny(ny)

In [85]:
len(ny)

198289

In [84]:
ny.isna().sum()

camis                    0
dba                      0
boro                     0
building                 0
street                   0
zipcode                  0
phone                    0
inspection_date          0
critical_flag            0
record_date              0
latitude                 0
longitude                0
cuisine_description      0
action                   0
violation_code           0
violation_description    0
score                    0
grade                    0
inspection_type          0
dtype: int64

In [109]:
ny = a.acquire_ny()
ny = prep.clean_ny(ny)

In [110]:
ny.isna().sum()

camis                    0
dba                      0
boro                     0
building                 0
street                   0
zipcode                  0
phone                    0
inspection_date          0
record_date              0
latitude                 0
longitude                0
cuisine_description      0
action                   0
violation_code           0
violation_description    0
score                    0
grade                    0
inspection_type          0
dtype: int64

In [111]:
ny.nunique()

camis                    25820
dba                      20593
boro                         5
building                  7249
street                    2256
zipcode                    221
phone                    23872
inspection_date           1613
record_date                  2
latitude                 21802
longitude                21802
cuisine_description         89
action                       4
violation_code              73
violation_description      151
score                      130
grade                        3
inspection_type             27
dtype: int64

In [112]:
full_addy = ny.building + ' ' + ny.street + ' ' + ny.zipcode.astype(str)
ny['full_address'] = full_addy

In [113]:
ny

,camis,dba,boro,building,street,zipcode,phone,inspection_date,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,grade,inspection_type,full_address
2,41168748,DUNKIN,Bronx,880,GARRISON AVENUE,10474,7188614171,2022-03-30T00:00:00.000,2023-10-26T06:00:11.000,40.816753,-73.892364,Donuts,Violations cited,10J,Hand wash sign not posted,13.0,A,Cycle Inspection / Initial Inspection,880 GARRISON AVENUE 10474
6,41688142,TABLE 87,Brooklyn,620,ATLANTIC AVENUE,11217,9176186100,2017-01-25T00:00:00.000,2023-10-26T06:00:11.000,40.683447,-73.975691,Pizza,No violations,No violation,No violation,0.0,A,Cycle Inspection / Initial Inspection,620 ATLANTIC AVENUE 11217
18,50100336,SUBWAY,Brooklyn,8711,3 AVENUE,11209,7186808808,2022-04-05T00:00:00.000,2023-10-26T06:00:11.000,40.622569,-74.031412,Sandwiches,Violations cited,09B,Thawing procedures improper.,10.0,A,Cycle Inspection / Initial Inspection,8711 3 AVENUE 11209
21,50086686,GERTIE,Brooklyn,58,MARCY AVENUE,11211,7186360902,2021-08-25T00:00:00.000,2023-10-26T06:00:13.000,40.712360,-73.955419,American,No violations,No violation,No violation,0.0,A,Cycle Inspection / Initial Inspection,58 MARCY AVENUE 11211
24,50081121,DUNKIN,Brooklyn,2492,LINDEN BOULEVARD,11208,7182729090,2022-04-04T00:00:00.000,2023-10-26T06:00:11.000,40.666827,-73.871606,Donuts,Violations cited,10J,Hand wash sign not posted,24.0,B,Cycle Inspection / Initial Inspection,2492 LINDEN BOULEVARD 11208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207924,41571471,LITTLE CAESARS,Queens,14605,JAMAICA AVENUE,11435,7182973450,2022-10-21T00:00:00.000,2023-10-26T06:00:11.000,40.701806,-73.808704,Pizza,Violations cited,09B,Thawing procedure improper.,7.0,A,Cycle Inspection / Initial Inspection,14605 JAMAICA AVENUE 11435
207925,50130884,NORA PIZZA,Bronx,2456,EASTCHESTER ROAD,10469,6464921148,2023-05-15T00:00:00.000,2023-10-26T06:00:11.000,40.861919,-73.843318,Pizza,Violations cited,02G,Cold TCS food item held above 41 °F; smoked or...,26.0,B,Pre-permit (Operational) / Re-inspection,2456 EASTCHESTER ROAD 10469
207926,41139255,HUDSON FOOD COURT,Manhattan,333,HUDSON STREET,10013,2123672192,2022-09-06T00:00:00.000,2023-10-26T06:00:11.000,40.727022,-74.007378,American,Violations cited,08A,Establishment is not free of harborage or cond...,0.0,A,Cycle Inspection / Re-inspection,333 HUDSON STREET 10013
207927,50011932,ANTICA RISTORANTE,Manhattan,8,STONE STREET,10004,2124803880,2019-11-25T00:00:00.000,2023-10-26T06:00:11.000,40.704018,-74.012638,Italian,Violations cited,04M,Live roaches present in facility's food and/or...,48.0,C,Cycle Inspection / Re-inspection,8 STONE STREET 10004


In [117]:
ny.inspection_date.nunique()

1613

In [118]:
agg_violations = ny.groupby(['camis','inspection_date']).agg({'violation_code': lambda x: x.tolist(),'violation_description':lambda x: x.tolist()})

In [156]:
agg_violations.violation_code[:80]

camis     inspection_date        
30075445  2021-08-10T00:00:00.000                                       [06E]
          2023-01-31T00:00:00.000    [04H, 08C, 04L, 10D, 10F, 06C, 04N, 06D]
          2023-02-03T00:00:00.000                                  [10F, 02G]
          2023-08-01T00:00:00.000                             [08A, 05H, 04L]
          2023-08-22T00:00:00.000                             [04L, 08A, 08C]
                                                       ...                   
40364439  2022-05-18T00:00:00.000                        [04L, 08C, 06C, 08A]
          2023-05-17T00:00:00.000                             [10B, 04L, 08A]
40364443  2021-07-30T00:00:00.000                             [02G, 08A, 04L]
          2022-01-13T00:00:00.000                   [08C, 10B, 04L, 08A, 02G]
          2022-08-08T00:00:00.000                        [08A, 04L, 02G, 10F]
Name: violation_code, Length: 80, dtype: object

In [141]:
agg_violations.loc[(30075445, '2021-08-10T00:00:00.000')]

violation_code                                                       [06E]
violation_description    [Sanitized equipment or utensil, including in-...
Name: (30075445, 2021-08-10T00:00:00.000), dtype: object

In [130]:
zip(ny.camis, ny.inspection_date)

In [160]:
agg_data_code = []
agg_data_description = []

for cam, date in zip(ny.camis, ny.inspection_date):
    agg_data_code.append(agg_violations.loc[(cam, date)][0])
    agg_data_description.append(agg_violations.loc[(cam, date)][1])
    
ny.violation_code = agg_data_code
ny.violation_description = agg_data_description

In [169]:
ny

,camis,dba,boro,building,street,zipcode,phone,inspection_date,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,grade,inspection_type,full_address
2,41168748,DUNKIN,Bronx,880,GARRISON AVENUE,10474,7188614171,2022-03-30T00:00:00.000,2023-10-26T06:00:11.000,40.816753,-73.892364,Donuts,Violations cited,"[10J, 04N, 08A]","[Hand wash sign not posted, Filth flies or foo...",13.0,A,Cycle Inspection / Initial Inspection,880 GARRISON AVENUE 10474
6,41688142,TABLE 87,Brooklyn,620,ATLANTIC AVENUE,11217,9176186100,2017-01-25T00:00:00.000,2023-10-26T06:00:11.000,40.683447,-73.975691,Pizza,No violations,[No violation],[No violation],0.0,A,Cycle Inspection / Initial Inspection,620 ATLANTIC AVENUE 11217
18,50100336,SUBWAY,Brooklyn,8711,3 AVENUE,11209,7186808808,2022-04-05T00:00:00.000,2023-10-26T06:00:11.000,40.622569,-74.031412,Sandwiches,Violations cited,"[09B, 10F, 06D]","[Thawing procedures improper., Non-food contac...",10.0,A,Cycle Inspection / Initial Inspection,8711 3 AVENUE 11209
21,50086686,GERTIE,Brooklyn,58,MARCY AVENUE,11211,7186360902,2021-08-25T00:00:00.000,2023-10-26T06:00:13.000,40.712360,-73.955419,American,No violations,[No violation],[No violation],0.0,A,Cycle Inspection / Initial Inspection,58 MARCY AVENUE 11211
24,50081121,DUNKIN,Brooklyn,2492,LINDEN BOULEVARD,11208,7182729090,2022-04-04T00:00:00.000,2023-10-26T06:00:11.000,40.666827,-73.871606,Donuts,Violations cited,"[10J, 02G, 04N, 10F, 08A]","[Hand wash sign not posted, Cold food item hel...",24.0,B,Cycle Inspection / Initial Inspection,2492 LINDEN BOULEVARD 11208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207924,41571471,LITTLE CAESARS,Queens,14605,JAMAICA AVENUE,11435,7182973450,2022-10-21T00:00:00.000,2023-10-26T06:00:11.000,40.701806,-73.808704,Pizza,Violations cited,"[10G, 09B]",[Dishwashing and ware washing: Cleaning and s...,7.0,A,Cycle Inspection / Initial Inspection,14605 JAMAICA AVENUE 11435
207925,50130884,NORA PIZZA,Bronx,2456,EASTCHESTER ROAD,10469,6464921148,2023-05-15T00:00:00.000,2023-10-26T06:00:11.000,40.861919,-73.843318,Pizza,Violations cited,"[02B, 04L, 08A, 02G]",[Hot TCS food item not held at or above 140 °F...,26.0,B,Pre-permit (Operational) / Re-inspection,2456 EASTCHESTER ROAD 10469
207926,41139255,HUDSON FOOD COURT,Manhattan,333,HUDSON STREET,10013,2123672192,2022-09-06T00:00:00.000,2023-10-26T06:00:11.000,40.727022,-74.007378,American,Violations cited,"[04N, 02G, 06C, 10C, 10F, 10B, 08A]",[Filth flies or food/refuse/sewage associated ...,0.0,A,Cycle Inspection / Re-inspection,333 HUDSON STREET 10013
207927,50011932,ANTICA RISTORANTE,Manhattan,8,STONE STREET,10004,2124803880,2019-11-25T00:00:00.000,2023-10-26T06:00:11.000,40.704018,-74.012638,Italian,Violations cited,"[10B, 10F, 05C, 06C, 08A, 04M]",[Plumbing not properly installed or maintained...,48.0,C,Cycle Inspection / Re-inspection,8 STONE STREET 10004


In [172]:
ny[ny.dba == '$1 PIZZA'].sort_values('inspection_date')

,camis,dba,boro,building,street,zipcode,phone,inspection_date,record_date,latitude,longitude,cuisine_description,action,violation_code,violation_description,score,grade,inspection_type,full_address
153929,50086385,$1 PIZZA,Manhattan,832,6 AVENUE,10001,9175878888,2022-02-22T00:00:00.000,2023-10-26T06:00:11.000,40.746510,-73.990158,Pizza,Violations cited,[02B],[Hot food item not held at or above 140º F.],7.0,A,Cycle Inspection / Re-inspection,832 6 AVENUE 10001
170198,50117047,$1 PIZZA,Manhattan,333B,AVENUE OF THE AMERICAS,10014,9173883956,2022-05-04T00:00:00.000,2023-10-26T06:00:11.000,40.731409,-74.001176,Pizza,Violations cited,"[06B, 02B, 04A, 06C]","[Tobacco use, eating, or drinking from open co...",0.0,A,Pre-permit (Operational) / Initial Inspection,333B AVENUE OF THE AMERICAS 10014
157885,50117047,$1 PIZZA,Manhattan,333B,AVENUE OF THE AMERICAS,10014,9173883956,2022-05-04T00:00:00.000,2023-10-26T06:00:11.000,40.731409,-74.001176,Pizza,Violations cited,"[06B, 02B, 04A, 06C]","[Tobacco use, eating, or drinking from open co...",0.0,A,Pre-permit (Operational) / Initial Inspection,333B AVENUE OF THE AMERICAS 10014
49822,50117047,$1 PIZZA,Manhattan,333B,AVENUE OF THE AMERICAS,10014,9173883956,2022-05-04T00:00:00.000,2023-10-26T06:00:11.000,40.731409,-74.001176,Pizza,Violations cited,"[06B, 02B, 04A, 06C]","[Tobacco use, eating, or drinking from open co...",0.0,A,Pre-permit (Operational) / Initial Inspection,333B AVENUE OF THE AMERICAS 10014
156271,50117047,$1 PIZZA,Manhattan,333B,AVENUE OF THE AMERICAS,10014,9173883956,2022-05-04T00:00:00.000,2023-10-26T06:00:11.000,40.731409,-74.001176,Pizza,Violations cited,"[06B, 02B, 04A, 06C]","[Tobacco use, eating, or drinking from open co...",0.0,A,Pre-permit (Operational) / Initial Inspection,333B AVENUE OF THE AMERICAS 10014
139377,50117047,$1 PIZZA,Manhattan,333B,AVENUE OF THE AMERICAS,10014,9173883956,2022-06-06T00:00:00.000,2023-10-26T06:00:11.000,40.731409,-74.001176,Pizza,Violations cited,"[02G, 10F]",[Cold food item held above 41º F (smoked fish ...,0.0,A,Pre-permit (Operational) / Re-inspection,333B AVENUE OF THE AMERICAS 10014
41202,50117047,$1 PIZZA,Manhattan,333B,AVENUE OF THE AMERICAS,10014,9173883956,2022-06-06T00:00:00.000,2023-10-26T06:00:11.000,40.731409,-74.001176,Pizza,Violations cited,"[02G, 10F]",[Cold food item held above 41º F (smoked fish ...,0.0,A,Pre-permit (Operational) / Re-inspection,333B AVENUE OF THE AMERICAS 10014
38621,50086385,$1 PIZZA,Manhattan,832,6 AVENUE,10001,9175878888,2022-08-16T00:00:00.000,2023-10-26T06:00:11.000,40.746510,-73.990158,Pizza,Violations cited,"[10B, 08A, 10D, 02G, 06C, 04N]",[Anti-siphonage or back-flow prevention device...,25.0,B,Cycle Inspection / Initial Inspection,832 6 AVENUE 10001
92927,50086385,$1 PIZZA,Manhattan,832,6 AVENUE,10001,9175878888,2022-08-16T00:00:00.000,2023-10-26T06:00:11.000,40.746510,-73.990158,Pizza,Violations cited,"[10B, 08A, 10D, 02G, 06C, 04N]",[Anti-siphonage or back-flow prevention device...,25.0,B,Cycle Inspection / Initial Inspection,832 6 AVENUE 10001
94271,50086385,$1 PIZZA,Manhattan,832,6 AVENUE,10001,9175878888,2022-08-16T00:00:00.000,2023-10-26T06:00:11.000,40.746510,-73.990158,Pizza,Violations cited,"[10B, 08A, 10D, 02G, 06C, 04N]",[Anti-siphonage or back-flow prevention device...,25.0,B,Cycle Inspection / Initial Inspection,832 6 AVENUE 10001


In [166]:
ny.groupby(['camis','inspection_date'])

In [159]:
len(agg_data_code)

198289

In [139]:
agg_violations.index[0]

(30075445, '2021-08-10T00:00:00.000')

In [137]:
agg_data[0]

(41168748, '2022-03-30T00:00:00.000')

In [94]:
ny.groupby('dba').camis.nunique()

dba
"U" LIKE CHINESE TAKE OUT           1
"W" CAFE                            1
#1 GARDEN CHINESE RESTAURANT        1
#1 SABOR LATINO RESTAURANT          1
$1 PIZZA                            2
                                   ..
jackies best italian pizza llc      1
kampai hibachi & brazilian Grill    1
kokomo restaurant                   1
matchpointnyc                       1
mukbang                             1
Name: camis, Length: 20593, dtype: int64